<a href="https://colab.research.google.com/github/efriedli/zero-inbox/blob/main/ZeroInbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero Inbox

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates Zero Inbox's server side capabilities: parsing emails to get actions, events and recaps.

In [ ]:
!pip install -U -q google.generativeai # Install the Python SDK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

## Set up API key

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel(
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        max_output_tokens=8192,
        temperature=0.9,
    ))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/zero-inbox-resources/client_secret_278932902519-dsv6fmo2239mqv7h4gnih5o97pjvcbab.apps.googleusercontent.com.json"

/content/drive/MyDrive/zero-inbox-resources/client_secret_278932902519-dsv6fmo2239mqv7h4gnih5o97pjvcbab.apps.googleusercontent.com.json


In [ ]:
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from googleapiclient import errors
import os
import pickle

# Path to your credentials file
CREDENTIALS_FILE = '/content/drive/MyDrive/zero-inbox-resources/client_secret_278932902519-dsv6fmo2239mqv7h4gnih5o97pjvcbab.apps.googleusercontent.com.json'

# Scopes required by Gmail
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail_api():
    creds = None
    # Load the saved credentials if they exist
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no valid credentials, let the user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = Flow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES, redirect_uri='urn:ietf:wg:oauth:2.0:oob')
            auth_url, _ = flow.authorization_url(prompt='consent')
            print('Please go to this URL and authorize the app:', auth_url)
            code = input('Enter the authorization code: ')
            flow.fetch_token(code=code)
            creds = flow.credentials
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def get_gmail_messages():
    creds = authenticate_gmail_api()
    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API to fetch the messages
    results = service.users().messages().list(userId='me').execute()
    messages = results.get('messages', [])

    if not messages:
        print('No messages found.')
    else:
        print('Message IDs:')
        for message in messages:
            print(message['id'])


# Retrieve and print message IDs
get_gmail_messages()


def get_contents():
    creds = authenticate_gmail_api()
    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API to fetch the messages
    try:
        message = service.users().messages().get(userId='me', id='18f00117a0689dd0').execute()
        print(F'Message snippet: {message["snippet"]}')
        return message
    except errors.HttpError as error:
        print(F'An error occurred: {error}')
        return None

# Retrieve one specific contents
get_contents()


Please go to this URL and authorize the app: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=278932902519-dsv6fmo2239mqv7h4gnih5o97pjvcbab.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=aqrui1xW41ddf2UXJoggcewoN5G1j7&prompt=consent&access_type=offline
Enter the authorization code: 4/1AeaYSHD_YkA7VawYyNsGeFECP8mTtim4OJgQH1IeCMibqaE7BRXR6kI9zN8
Message IDs:
18f00117a0689dd0
18f0005d314d4612
18efffc4fede34af
Message snippet: Google Drive for desktop was granted access to your Google Account jeff.skilling.zero@gmail.com If you did not grant access, you should check this activity and secure your account. Check activity You


{'id': '18f00117a0689dd0',
 'threadId': '18f00117a0689dd0',
 'labelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Google Drive for desktop was granted access to your Google Account jeff.skilling.zero@gmail.com If you did not grant access, you should check this activity and secure your account. Check activity You',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'jeff.skilling.zero@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6124:1e27:b0:39a:67af:1906 with SMTP id k39csp1482597vlr;        Sun, 21 Apr 2024 02:51:37 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a54:4516:0:b0:3c4:e370:8f70 with SMTP id l22-20020a544516000000b003c4e3708f70mr6690199oil.21.1713693096889;        Sun, 21 Apr 2024 02:51:36 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1713693096; cv=none;        d=google.com; s=arc-20160816;        b=u6By9QKvfe6iW

In [ ]:
from base64 import urlsafe_b64decode
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from email import message_from_bytes
import os
import pickle

def authenticate_gmail_api():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds = None  # Placeholder: Set up your actual authentication here
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def get_mime_message(service, user_id, msg_id):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id, format='raw').execute()
        msg_raw = urlsafe_b64decode(message['raw'].encode('ASCII'))
        mime_msg = message_from_bytes(msg_raw)
        email_body = ""
        if mime_msg.is_multipart():
            for part in mime_msg.walk():
                if part.get_content_type() == 'text/plain':
                    email_body = part.get_payload(decode=True).decode()
                    break  # Assuming we only need the first text/plain part
        else:
            email_body = mime_msg.get_payload(decode=True).decode()
        return email_body
    except Exception as error:
        print('An error occurred:', error)
        return None

# Usage example
creds = authenticate_gmail_api()
if creds:
    service = build('gmail', 'v1', credentials=creds)
    email_body = get_mime_message(service, 'me', '18f00117a0689dd0')
    if email_body:
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content("Summarize this email in a few words: " + email_body)
        print(response.text)

Google Drive for desktop granted access to Gmail account.


In [ ]:
# prompt: print the contents of my Google Drive file stored at zero-inbox-resources/emailParseStructure.json

import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the file
file_path = '/content/drive/MyDrive/zero-inbox-resources/emailParseStructure.json'

# Read the file contents
with open(file_path, 'r') as f:
  data = json.load(f)

# Print the file contents
print(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'totalEmails': {'type': 'integer', 'description': 'Total number of emails in the inbox'}, 'categories': {'actionRequired': {'type': 'array', 'items': {'$ref': '#/definitions/actionableEmail'}}, 'dailySummary': {'type': 'object', 'properties': {'date': {'type': 'string', 'format': 'date', 'description': 'Date of the summary'}, 'summaryItems': {'type': 'array', 'items': {'$ref': '#/definitions/summaryItem'}}}, 'required': ['date', 'summaryItems']}, 'events': {'type': 'array', 'items': {'$ref': '#/definitions/eventDetails'}}}, 'additionalProperties': False}, 'definitions': {'emailSummary': {'type': 'object', 'properties': {'subject': {'type': 'string', 'description': 'The subject line of the email'}, 'sender': {'type': 'string', 'description': 'The email address of the sender

In [ ]:
from base64 import urlsafe_b64decode
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from email import message_from_bytes
import os
import pickle
import json
from google.colab import drive

def authenticate_gmail_api():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds = None  # Placeholder: Set up your actual authentication here
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def get_mime_message(service, user_id, msg_id):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id, format='raw').execute()
        msg_raw = urlsafe_b64decode(message['raw'].encode('ASCII'))
        mime_msg = message_from_bytes(msg_raw)
        email_body = ""
        if mime_msg.is_multipart():
            for part in mime_msg.walk():
                if part.get_content_type() == 'text/plain':
                    email_body = part.get_payload(decode=True).decode()
                    break  # Assuming we only need the first text/plain part
        else:
            email_body = mime_msg.get_payload(decode=True).decode()
        return email_body
    except Exception as error:
        print('An error occurred:', error)
        return None


# Usage example
creds = authenticate_gmail_api()
if creds:
    service = build('gmail', 'v1', credentials=creds)
    email_body = get_mime_message(service, 'me', '18f00117a0689dd0')
    if email_body:
        # Path to the file
        file_path = '/content/drive/MyDrive/zero-inbox-resources/emailParseStructure.json'

        # Read the file contents
        with open(file_path, 'r') as f:
            jsonSchema = json.load(f)

        # Convert JSON schema to a string format that can be included in the prompt
        json_schema_str = json.dumps(jsonSchema, indent=2)

        # Gemini prompt
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content(
            "You are a virtual assistant who will help someone filter out the content of its mailbox. "
            "We want you to reduce noise as much as possible so the user will get directly into data, knowledge, and actions. "
            "Parse the content of this email: " + email_body + " "
            "according to this JSON schema: " + json_schema_str)  # Ensure concatenation is done with strings
        print(response.text)

```json
{
  "totalEmails": 1,
  "categories": {
    "actionRequired": [
      {
        "subject": "Google Drive for desktop was granted access to your Google Account",
        "actionRequired": "Check activity to make sure the access is legitimate and secure your account",
        "deadline": null
      }
    ],
    "dailySummary": [],
    "events": []
  }
}
```


In [ ]:
from base64 import urlsafe_b64decode
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from email import message_from_bytes
import os
import pickle
import json
import google.ai.generativelanguage as glm


# Path to the file
file_path = '/content/drive/MyDrive/zero-inbox-resources/emailParseStructure.json'

# Read the file contents
with open(file_path) as file:
  json_schema = file.read()

# Path to the file
file_path = '/content/drive/MyDrive/zero-inbox-resources/skilling_emails_light.txt'

# Read the file contents
with open(file_path) as file:
  emails_txt = file.read()

# Gemini prompt
model = genai.GenerativeModel('gemini-pro')

prompt = "You are a virtual assistant named ZeroInbox. "
"Your task is to help Jeff efficiently filter and understand the content of his mailbox. "
"Follow these guidelines:"
"Direct Address: Speak directly to Jeff using the second person ('you') and address him by name."
"Actionable Information: Focus on providing clear, concise, and actionable information. Use active voice and keep sentences short."
"Reduce Noise: Eliminate unnecessary details and fluff. Prioritize data, knowledge, and actions."
"Concise Output: Limit the total number of extracted elements to a maximum of 15."
"Steps:"
"- read the content of the inbox"
"- export following the JSON Schema"

response = model.generate_content([prompt, json_schema, emails_txt])

#prompt = "You are a virtual assistant tasked with filtering email content."
#"Reduce noise and focus on key data, knowledge, and actions."

#response = model.generate_content([prompt, combined_email_body])

print(response.text)

{
    "totalEmails": 66,
    "categories": {
        "actionRequired": [
            {
                "subject": "ERV Notification:  (Violation/Notification Memo - 10/19/2001)",
                "sender": "kenneth.thibodeaux@enron.com",
                "dateReceived": "10/19/2001",
                "summary": "PRELIMINARY"
            },
            {
                "subject": "ERV Notification:  (Violation/Notification Memo - 09/12/2001)",
                "sender": "kenneth.thibodeaux@enron.com",
                "dateReceived": "09/12/2001",
                "summary": null
            },
            {
                "subject": "ERV Notification:  (Violation/Notification Memo - 10/24/2001)",
                "sender": "kenneth.thibodeaux@enron.com",
                "dateReceived": "10/24/2001",
                "summary": "PRELIMINARY"
            },
            {
                "subject": "ERV Notification:  (Violation/Notification Memo - 09/27/2001)",
                "sender": "kenne

In [ ]:
!pip install jinja2

In [ ]:
# once Gemini outputs the json, merge it with our html template (server side rendering)
# and send to the user
import json
from jinja2 import Environment, FileSystemLoader

def load_json_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def render_html_from_json(json_data, template_name='template.html'):
    # Load templates from the current directory
    env = Environment(loader=FileSystemLoader('.'))
    # Load the specified template file
    template = env.get_template(template_name)
    # Render HTML content using the JSON data
    return template.render(data=json_data)

def main():
    # Load JSON data from file
    json_data = load_json_data('content.json')
    # Render HTML from the JSON data using a template
    html_output = render_html_from_json(json_data)

    # Write the output to an HTML file
    with open('index.html', 'w') as file:
        file.write(html_output)

if __name__ == "__main__":
    main()